In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok opencv-python onnxruntime tensorflow
!pip install python-multipart
!ngrok config add-authtoken 2w632UmskfhkiCHNqk8koimwg08_89pfMfZsEGcJj2q56zHkD

Authtoken saved to configuration file: C:\Users\Fayroz\AppData\Local/ngrok/ngrok.yml


In [2]:
import cv2
import numpy as np
import os
import json
import shutil
from fastapi import FastAPI, UploadFile
from fastapi.responses import FileResponse, JSONResponse
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import onnxruntime as ort
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [3]:
app = FastAPI()

# Load Models
feature_extractor = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
action_model = load_model('tennis_action_model.h5')
movenet_session = ort.InferenceSession('movenet_int8.onnx')

In [4]:
SEQUENCE_LENGTH = 10
IMG_SIZE = (299, 299)

labels = [
    'backhand', 'backhand2hands', 'backhand_slice', 'backhand_volley',
    'flat_service', 'forehand_flat', 'forehand_openstands', 'forehand_slice',
    'forehand_volley', 'kick_service', 'slice_service', 'smash'
]


In [6]:
def extract_frames(video_file, max_frames=SEQUENCE_LENGTH):
    cap = cv2.VideoCapture(video_file)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // max_frames)
    frames = []

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, IMG_SIZE)
        frame = preprocess_input(img_to_array(frame))
        frames.append(frame)
        if len(frames) == max_frames:
            break
    cap.release()
    while len(frames) < max_frames:
        frames.append(np.zeros_like(frames[0]))
    return np.array(frames)

def decode_prediction(pred):
    return labels[np.argmax(pred)]

def check_correctness(predicted_label):
    return True  # Placeholder: All detected movements are correct

def extract_joints_with_movenet(frame):
    input_size = 192
    resized_frame = cv2.resize(frame, (input_size, input_size))
    input_image = np.expand_dims(resized_frame, axis=0).astype(np.float32) / 255.0  # Normalize

    input_name = movenet_session.get_inputs()[0].name
    output_name = movenet_session.get_outputs()[0].name

    outputs = movenet_session.run([output_name], {input_name: input_image})
    heatmaps = outputs[0][0]  # Shape: (48, 48, 17)

    joint_names = [
        "nose", "left_eye", "right_eye", "left_ear", "right_ear",
        "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
        "left_wrist", "right_wrist", "left_hip", "right_hip",
        "left_knee", "right_knee", "left_ankle", "right_ankle"
    ]

    normalized_keypoints = []
    for i in range(heatmaps.shape[-1]):
        heatmap = heatmaps[:, :, i]
        y, x = np.unravel_index(np.argmax(heatmap), heatmap.shape)

        x_normalized = x / heatmaps.shape[1]
        y_normalized = y / heatmaps.shape[0]

        normalized_keypoints.append({
            "joint": joint_names[i],
            "x": float(round(x_normalized, 2)),
            "y": float(round(y_normalized, 2))
        })

    return normalized_keypoints


In [7]:
import os
from fastapi import FastAPI, UploadFile
from fastapi.responses import FileResponse, JSONResponse
import shutil
import json
import cv2
import numpy as np

app = FastAPI()

# Define a writable directory for temporary files
OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create the directory if it doesn't exist

@app.post("/analyze_video/")
async def analyze_video(file: UploadFile):
    try:
        # Save the uploaded video to a temporary file
        temp_video_path = os.path.join(OUTPUT_DIR, "temp_video.mp4")
        with open(temp_video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # Extract frames from the video
        frames = extract_frames(temp_video_path)  # Shape: (10, 299, 299, 3)

        # Extract features from frames
        sequence_features = []
        for frame in frames:
            frame_exp = np.expand_dims(frame, axis=0)
            features = feature_extractor.predict(frame_exp, verbose=0)
            sequence_features.append(features[0])

        sequence_features = np.array(sequence_features)  # Shape: (10, 2048)
        sequence_features = np.expand_dims(sequence_features, axis=0)  # Shape: (1, 10, 2048)

        # Predict the action
        prediction = action_model.predict(sequence_features)
        predicted_label = decode_prediction(prediction)

        # Extract joints from the last frame
        raw_frame = cv2.VideoCapture(temp_video_path)
        raw_frame.set(cv2.CAP_PROP_POS_FRAMES, int(raw_frame.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
        ret, last_frame = raw_frame.read()
        raw_frame.release()

        joints = extract_joints_with_movenet(last_frame)

        # Prepare the response
        response = {
            "player": {
                "isCorrect": check_correctness(predicted_label),
                "movement": predicted_label,
                "prediction": predicted_label,
                "joints": joints
            }
        }

        # Save the response to a JSON file
        output_file_path = os.path.join(OUTPUT_DIR, "player_analysis.json")
        with open(output_file_path, "w") as outfile:
            json.dump(response, outfile, indent=2)

        # Return the JSON file as a response
        return FileResponse(path=output_file_path, media_type='application/json', filename="player_analysis.json")

    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [ ]:
nest_asyncio.apply()

public_url = ngrok.connect(8000)
print(f"API is running at: {public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:pyngrok.process.ngrok:t=2025-04-23T14:06:52+0200 lvl=eror msg="unable to evaluate ngrok agent binary path for symlinks" obj=tunnels.session err="CreateFile C:\\Users\\Fayroz\\AppData\\Local\\ngrok\\ngrok.exe: The system cannot find the file specified."


API is running at: NgrokTunnel: "https://b180-154-178-69-113.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [24208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
